In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.2 MB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 42.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 21.3 MB/s eta 0:00:01


In [12]:
"""
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
# model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
PATH_model = '../Model/output/T5 Model/t5-base-finetuned-summarize-news.pth'
PATH_tokenizer = '../Model/output/T5 Model/t5-tokenizer-summarize-news.pth'

torch.save(model, PATH_model)
torch.save(tokenizer, PATH_tokenizer)
"""

In [2]:
import torch

PATH_model = '../Model/output/T5 Model/t5-base-finetuned-summarize-news.pth'
PATH_tokenizer = '../Model/output/T5 Model/t5-tokenizer-summarize-news.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.load(PATH_model).to(device)
tokenizer = torch.load(PATH_tokenizer)

def summarize(text, max_length=150):
    # input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True).to(device)
    
    encoding = tokenizer(
    text,
    padding="longest",
    max_length=1500,
    truncation=True,
    return_tensors="pt",
    )
    
    # input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
    input_ids = encoding.input_ids
    input_ids = input_ids.to(device)
    generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

    return preds[0]

In [3]:
import pandas as pd
df_test = pd.read_csv('Dataset/generated_article_8078.csv') 
# df = df_test.iloc[:3000]
df = df_test
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8078 entries, 0 to 8077
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           8078 non-null   int64 
 1   Real title      8078 non-null   object
 2   Generated text  8078 non-null   object
 3   label           8078 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 252.6+ KB


In [4]:
from datetime import datetime
START = datetime.now()
print(START)
i = 1
c = 0
list = []
for index, row in df.iterrows():
    d = {}
    try:
        title = row['Real title']
        text = row['Generated text'].replace(title, '')
        output = summarize(text, max_length=150)
        c += 1
        if i%300 == 0:
            print(str(i)+'/'+str(len(df)),' || Time: ' + str(datetime.now() - START), 'Total steps: ' + str(c))

        try:
            output = output.split('nnn')[0]
        except:
            output = output
        text = title  + output
        d['index'] = row['index']
        d['title'] = title
        d['text'] = text
        d['label'] = row['label']
        list.append(d)
        i += 1
    except:
        pass
print(datetime.now() - START)

2022-06-03 13:07:17.489383
300/8078  || Time: 0:11:36.895959 Total steps: 300
600/8078  || Time: 0:23:03.539388 Total steps: 600
900/8078  || Time: 0:35:27.215656 Total steps: 900
1200/8078  || Time: 0:47:38.568674 Total steps: 1200
1500/8078  || Time: 1:00:00.856950 Total steps: 1500
1800/8078  || Time: 1:12:13.955946 Total steps: 1800
2100/8078  || Time: 1:24:32.605471 Total steps: 2100
2400/8078  || Time: 1:36:38.241870 Total steps: 2400
2700/8078  || Time: 1:48:36.825060 Total steps: 2700
3000/8078  || Time: 2:00:40.348127 Total steps: 3000
3300/8078  || Time: 2:12:20.275385 Total steps: 3300
3600/8078  || Time: 2:24:41.260895 Total steps: 3600
3900/8078  || Time: 2:36:57.198686 Total steps: 3900
4200/8078  || Time: 2:48:30.719001 Total steps: 4200
4500/8078  || Time: 3:00:24.554808 Total steps: 4500
4800/8078  || Time: 3:12:35.663935 Total steps: 4800
5100/8078  || Time: 3:24:39.890263 Total steps: 5100
5400/8078  || Time: 3:36:52.860735 Total steps: 5400
5700/8078  || Time: 3:48:

In [ ]:
PATH = 'summarized-article/summarize-gene01.csv'
pd.DataFrame(list).to_csv(PATH, index=False)

In [ ]:
pp = pd.read_csv(PATH)
pp

8078